In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split


from tqdm import tqdm

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import re
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Bidirectional,LSTM,Activation,Conv1D,MaxPool1D,Dropout
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.utils import np_utils
np.random.seed(1)

%matplotlib inline


In [ ]:
t = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/train.tsv.zip', sep = '\t')
t.head()

In [ ]:
te = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/test.tsv.zip',sep = '\t')
te.head()

In [ ]:
print("Train Data\n")
print(t.info())
print("***************************************")
print("Test Data\n")
te.info()

In [ ]:
t['Sentiment'].value_counts()

In [ ]:
val_array = t['Sentiment'].value_counts().to_xarray()
plt.bar(val_array.index, val_array)
plt.xlabel('review rating/Sentiments')
plt.ylabel('No of reviews')
plt.show()

In [ ]:
def sentence_cleaning(df):
    sentence = []
    for sent in tqdm(df['Phrase']):
        
        text = re.sub("[^a-zA-Z]"," ",sent)
        
        word = word_tokenize(text.lower())
        
        lemmatizer = WordNetLemmatizer()
        
        lemm_word = [lemmatizer.lemmatize(i) for i in word]
        
        sentence.append(lemm_word)
    return (sentence)

In [ ]:
t_sent = sentence_cleaning(t)
te_sent = sentence_cleaning(te)

print(len(t_sent))
print(len(te_sent))

In [ ]:
y_target = to_categorical(t['Sentiment'].values)

y_target.shape



In [ ]:
X_train,X_val,y_train,y_val = train_test_split(t_sent,y_target,test_size = 0.2,stratify = y_target)

In [ ]:
unique_words = set()
max_len = 0
for sent in tqdm(X_train):
    unique_words.update(sent)
    if(max_len < len(sent)):
        max_len = len(sent)
        sentence = sent

In [ ]:
print(t['Phrase'][0])
print((" ").join(t_sent[0]))

In [ ]:


y_target = to_categorical(t['Sentiment'].values)

y_target.shape



In [ ]:
print(t['Sentiment'].unique())
print(t['Sentiment'].nunique())
t['Sentiment'].value_counts()

In [ ]:
val_array = t['Sentiment'].value_counts().to_xarray()
plt.bar(val_array.index, val_array)
plt.xlabel('review rating/Sentiments')
plt.ylabel('No of reviews')
plt.show()


In [ ]:
%%time
t['Length'] = t['Phrase'].apply(lambda x: len(str(x).split(' ')))
import seaborn as sns
sns.pairplot(t,hue='Sentiment',vars=['PhraseId','SentenceId','Length'])

In [ ]:
def clean_sentences(df):
    
    reviews = []
    for sent in tqdm(df['Phrase']):
        # removing non-alphabetical characters 
        text = re.sub("[^a-zA-Z]"," ",sent)
        
        # Now tokenizing the sentence : 
        words = word_tokenize(text.lower())
        
        #removing stop words :
        new_words = [ ele for ele in words if ele.lower() not in stopwords.words('english') ]
        
        # Lemmatizing each word to its lemma
        lem = WordNetLemmatizer()
        lem_words = [lem.lemmatize(i) for i in new_words]
        
        #finally
        reviews.append(lem_words)
        
    return(reviews)

In [ ]:
%%time
train_sentences = clean_sentences(t)
test_sentences = clean_sentences(te)

print(len(train_sentences))
print(len(test_sentences))

In [ ]:
print(t['Phrase'][0])
print((" ").join(train_sentences[0]))


In [ ]:
y_target = to_categorical(t['Sentiment'].values)
y_target.shape

In [ ]:

X_train,X_val,y_train,y_val = train_test_split(train_sentences,y_target,test_size = 0.2,stratify = y_target)

unique_words = set()
len_max = -1

for sent in tqdm(X_train):
    unique_words.update(sent)
    if(len_max < len(sent)):
        len_max = len(sent)

print('Words in vocab : ' , len(list(unique_words)))
print('Max_length : ' , len_max)



In [ ]:
v = len(list(unique_words))
embedding_dim = 300
max_length = len_max
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'



In [ ]:
%%time
tokenizer = Tokenizer(num_words = v,
                      # filters = '#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      oov_token = oov_tok,
                      # lower = True,
                      char_level = False)

tokenizer.fit_on_texts(list(X_train))

# Training
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train,
                        maxlen = max_length,
                        padding = padding_type,
                        truncating = trunc_type)

# Validation
X_val = tokenizer.texts_to_sequences(X_val)
X_val = pad_sequences(X_val,
                      maxlen = max_length,
                      padding = padding_type,
                      truncating = trunc_type)

# Testing
X_test = tokenizer.texts_to_sequences(test_sentences)
X_test = pad_sequences(X_test,
                       maxlen = max_length,
                       padding = padding_type,
                       truncating = trunc_type)

In [ ]:
print("X_training shape   : ",X_train.shape)
print("X_validation shape : ",X_val.shape)
print("X_testing shape    : ",X_test.shape)

In [ ]:
model = Sequential()
model.add(Embedding(v,embedding_dim,input_length = max_length))
model.add(Bidirectional(LSTM(128,dropout = 0.2, recurrent_dropout = 0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(64, dropout = 0.2, recurrent_dropout = 0.2, return_sequences=False)))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(y_target.shape[1],activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
model.summary()

In [ ]:
early_stopping = EarlyStopping(min_delta = 0.001,
                               mode = 'max',
                               monitor = 'val_acc',
                               patience = 2)
callback = [early_stopping]



In [ ]:
%%time

num_epochs = 4

history = model.fit(X_train,y_train,
                    validation_data = (X_val, y_val),
                    epochs = num_epochs,
                    batch_size = 256,
                    verbose = 1,
                    callbacks = callback)



In [ ]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
    
plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

In [ ]:
test_id = te['PhraseId']

y_pred = np.argmax(model.predict(X_test), axis = -1)

submission_df = pd.DataFrame({'PhraseId': test_id, 'Sentiment': y_pred})
submission_df.to_csv('submission_.csv', index=False)
submission_df.head()

